<a href="https://colab.research.google.com/github/asiyabb/facial-expressions/blob/main/affectnet_efficientnet_b0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install timm pandas scikit-learn

import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import timm

from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix, classification_report

from google.colab import drive
drive.mount("/content/drive")

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

DATA_ROOT = Path("/content/drive/MyDrive/AffectNet")  # change if needed
TRAIN_DIR = DATA_ROOT / "train"
TEST_DIR  = DATA_ROOT / "test"
LABELS_CSV = DATA_ROOT / "labels.csv"

# Put the exact filename you saved
BEST_PATH = DATA_ROOT / "best_efficientnet_b0_affectnet.pt"
print("BEST_PATH exists:", BEST_PATH.exists(), BEST_PATH)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Device: cpu
BEST_PATH exists: True /content/drive/MyDrive/AffectNet/best_efficientnet_b0_affectnet.pt


In [3]:
CLASS_NAMES = ["Neutral","Happiness","Sadness","Surprise","Fear","Disgust","Anger","Contempt"]
NUM_CLASSES = len(CLASS_NAMES)

LABEL_SYNONYMS = {
    "neutral":"Neutral",
    "happiness":"Happiness", "happy":"Happiness",
    "sadness":"Sadness", "sad":"Sadness",
    "surprise":"Surprise",
    "fear":"Fear",
    "disgust":"Disgust",
    "anger":"Anger",
    "contempt":"Contempt",
}
name_to_id = {n: i for i, n in enumerate(CLASS_NAMES)}

df = pd.read_csv(LABELS_CSV)
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

df = df.rename(columns={"pth":"rel_path", "label":"raw_label"}).copy()
df["rel_path"] = df["rel_path"].astype(str).str.replace("\\", "/", regex=False).str.lstrip("/")

def to_label_id(x):
    s = str(x).strip().lower()
    if s in LABEL_SYNONYMS:
        return name_to_id[LABEL_SYNONYMS[s]]
    try:
        xi = int(x)
        if 0 <= xi < NUM_CLASSES:
            return xi
    except:
        pass
    return None

df["label"] = df["raw_label"].apply(to_label_id)
df = df[df["label"].notna()].copy()
df["label"] = df["label"].astype(int)

def resolve_path(rel_path: str):
    p_test = TEST_DIR / rel_path
    if p_test.exists():
        return p_test, "test"
    p_train = TRAIN_DIR / rel_path
    if p_train.exists():
        return p_train, "train"

    base = Path(rel_path).name
    p_test2 = TEST_DIR / base
    if p_test2.exists():
        return p_test2, "test"
    p_train2 = TRAIN_DIR / base
    if p_train2.exists():
        return p_train2, "train"
    return None, None

abs_paths, splits = [], []
for rp in df["rel_path"].tolist():
    p, sp = resolve_path(rp)
    abs_paths.append(str(p) if p else None)
    splits.append(sp)

df["abs_path"] = abs_paths
df["split"] = splits
df = df[df["abs_path"].notna()].copy().reset_index(drop=True)

test_df = df[df["split"] == "test"].copy().reset_index(drop=True)
print("Test images:", len(test_df))
test_df.head()


Test images: 11206


,rel_path,raw_label,relFCs,label,abs_path,split
0,disgust/ffhq_0.png,disgust,0.809775,5,/content/drive/MyDrive/AffectNet/test/disgust/...,test
1,disgust/ffhq_1.png,anger,0.830900,6,/content/drive/MyDrive/AffectNet/test/disgust/...,test
2,disgust/ffhq_10.png,disgust,0.831394,5,/content/drive/MyDrive/AffectNet/test/disgust/...,test
3,disgust/ffhq_11.png,disgust,0.737185,5,/content/drive/MyDrive/AffectNet/test/disgust/...,test
4,disgust/ffhq_12.png,disgust,0.819709,5,/content/drive/MyDrive/AffectNet/test/disgust/...,test


In [4]:
class ImgDataset(Dataset):
    def __init__(self, df, transform=None):
        self.paths = df["abs_path"].tolist()
        self.labels = df["label"].astype(int).tolist()
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img = Image.open(self.paths[idx]).convert("RGB")
        y = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, y

IMG_SIZE = 224  # must match what you trained with (224 in your code)
eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
])

test_ds = ImgDataset(test_df, transform=eval_tfms)

# KEY: num_workers=0 avoids Colab "child process" crash after idle
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

print("Test batches:", len(test_loader))


Test batches: 176


In [5]:
ckpt = torch.load(BEST_PATH, map_location=device)

# Use the saved model name if present, otherwise set manually
MODEL_NAME = ckpt.get("model_name", "efficientnet_b0")
print("MODEL_NAME:", MODEL_NAME)

model = timm.create_model(MODEL_NAME, pretrained=False, num_classes=NUM_CLASSES).to(device)

# Your checkpoint saved as {"state_dict": ...}
if "state_dict" not in ckpt:
    raise ValueError(f"Checkpoint keys: {list(ckpt.keys())} (expected 'state_dict')")

model.load_state_dict(ckpt["state_dict"])
model.eval()

criterion = nn.CrossEntropyLoss()

@torch.no_grad()
def evaluate(model, loader):
    all_y, all_p = [], []
    total_loss, n = 0.0, 0

    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        # no deprecated autocast warnings
        with torch.amp.autocast(device_type="cuda", enabled=(device=="cuda")):
            logits = model(x)
            loss = criterion(logits, y)

        total_loss += loss.item() * y.size(0)
        n += y.size(0)

        p = torch.argmax(logits, dim=1)
        all_y.append(y.detach().cpu().numpy())
        all_p.append(p.detach().cpu().numpy())

    all_y = np.concatenate(all_y)
    all_p = np.concatenate(all_p)
    acc = accuracy_score(all_y, all_p)
    bacc = balanced_accuracy_score(all_y, all_p)
    return total_loss / n, acc, bacc, all_y, all_p

te_loss, te_acc, te_bacc, te_y, te_p = evaluate(model, test_loader)

print("\nTEST:")
print("  loss:", te_loss)
print("  acc :", te_acc)
print("  bacc:", te_bacc)

print("\nConfusion Matrix (rows=true, cols=pred):")
print(confusion_matrix(te_y, te_p))

print("\nClassification Report:")
print(classification_report(te_y, te_p, target_names=CLASS_NAMES, digits=4))


MODEL_NAME: efficientnet_b0


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



TEST:
  loss: 0.9698917600867282
  acc : 0.7109584151347492
  bacc: 0.6754481352440304

Confusion Matrix (rows=true, cols=pred):
[[ 808   28   64   79   15   27   44   89]
 [  18 1971   18   40    1   36    4   71]
 [ 119   30 1096   23   20   49   39   23]
 [ 213   95   62 1466  187   63   35   44]
 [  41   14   87  288  953   53   39    8]
 [  63   94  123   58   47  979  102   43]
 [  76   11   71   48   26   84  354   20]
 [  88  163   12   15    1   17   11  340]]

Classification Report:
              precision    recall  f1-score   support

     Neutral     0.5666    0.7002    0.6264      1154
   Happiness     0.8192    0.9129    0.8635      2159
     Sadness     0.7149    0.7834    0.7476      1399
    Surprise     0.7268    0.6771    0.7011      2165
        Fear     0.7624    0.6426    0.6974      1483
     Disgust     0.7485    0.6488    0.6951      1509
       Anger     0.5637    0.5130    0.5372       690
    Contempt     0.5329    0.5255    0.5292       647

    accuracy 